# Imports

In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils
from importlib import reload 
import DQCAdapter as evg
from pyscf import gto, dft

## Init for DQC

In [2]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par

## Custom xc for PySCF

In [4]:
def custom_eval_xc(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vgamma = None
        vlapl = None
        vtau = None
        vxc = (vrho, vgamma, vlapl, vtau)
        fxc = None  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    return eval_xc

## Systems

In [5]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

h2_system = "H 0 0 0; H 1.4 0 0"

n2_system = "N 0 0 0; N 2.07 0 0"

ammonia = "N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721"

## Methods


In [25]:
def PySCF_calculation(system, a, p, verbose=0):
    xc = custom_eval_xc(a,p)
    mol = gto.M(atom = system, basis="6-31G", unit = 'Bohr')
    calc = dft.RKS(mol)
    calc = calc.define_xc_(xc, 'LDA')
    calc.verbose = verbose
    ene = calc.kernel()
    return ene

def DQC_calculation_tensors(system, a, p):
    atomzs, atomposs = dqc.parse_moldesc(system)
    mol = dqc.Mol((atomzs, atomposs), basis="6-31G")
    myxc = MyLDAX(a, p)
    qc = dqc.KS(mol, xc=myxc).run()
    return qc

def DQC_calculation(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    return DQC_calculation_tensors(system, a, p)

def finite_difference(system, a_value, p_value, coeff=0.000001):
    y = DQC_calculation(system, a_value, p_value).energy().detach()
    print("Energy DQC: ", y)
    da = a_value * coeff
    dp = p_value * coeff
    y_plus_dy_a = DQC_calculation(system, a_value+da, p_value)
    y_plus_dy_p = DQC_calculation(system, a_value, p_value+dp)
    dy_wrt_da = (y_plus_dy_a.energy().detach() - y) / da
    dy_wrt_dp = (y_plus_dy_p.energy().detach() - y) / dp
    return torch.tensor((dy_wrt_da, dy_wrt_dp))

def finite_difference_pyscf(system, a, p, coeff=0.000001, verbose = 3):
    y = PySCF_calculation(system, a, p, verbose)
    print("Energy PySCF: ", y)
    da = a * coeff
    dp = p * coeff
    y_plus_dy_a = PySCF_calculation(system, a+da, p)
    y_plus_dy_p = PySCF_calculation(system, a, p+dp)
    dy_wrt_da = (y_plus_dy_a - y) / da
    dy_wrt_dp = (y_plus_dy_p - y) / dp
    return torch.tensor((dy_wrt_da, dy_wrt_dp))


def reverse_mode(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    ene = DQC_calculation_tensors(system, a, p).energy()
    print("Energy DQC: ", ene.detach())
    grad_a, grad_p = torch.autograd.grad(ene, (a, p))
    return torch.tensor((grad_a, grad_p))

### Params a = 1.0, p = 2.0

In [26]:
print("REVERSE: ", reverse_mode(water, 1.0, 2.0))
print("FIN DQC: ", finite_difference(water, 1.0, 2.0))
print("FIN PySCF: ", finite_difference_pyscf(water, 1.0, 2.0))

Energy DQC:  tensor(-38.2050, dtype=torch.float64)
REVERSE:  tensor([ 9.9156, 19.0426], dtype=torch.float64)
Energy DQC:  tensor(-38.2050, dtype=torch.float64)
FIN DQC:  tensor([ 9.9156, 19.0426], dtype=torch.float64)
converged SCF energy = -38.2049675343314
Energy PySCF:  -38.204967534331374
FIN PySCF:  tensor([ 9.9156, 19.0425], dtype=torch.float64)


In [27]:
print("REVERSE: ", reverse_mode(h2_system, 1.0, 2.0))
print("FIN DQC: ", finite_difference(h2_system, 1.0, 2.0))
print("FIN PySCF: ", finite_difference_pyscf(h2_system, 1.0, 2.0))

Energy DQC:  tensor(-0.4216, dtype=torch.float64)
REVERSE:  tensor([ 0.0885, -0.2367], dtype=torch.float64)
Energy DQC:  tensor(-0.4216, dtype=torch.float64)
FIN DQC:  tensor([ 0.0885, -0.2367], dtype=torch.float64)
converged SCF energy = -0.421604471748425
Energy PySCF:  -0.42160447174842475
FIN PySCF:  tensor([ 0.0885, -0.2367], dtype=torch.float64)


In [28]:
print("REVERSE: ", reverse_mode(n2_system, 1.0, 2.0))
print("FIN DQC: ", finite_difference(n2_system, 1.0, 2.0))
print("FIN PySCF: ", finite_difference_pyscf(n2_system, 1.0, 2.0))

Energy DQC:  tensor(-54.3589, dtype=torch.float64)
REVERSE:  tensor([14.7676, 26.0215], dtype=torch.float64)
Energy DQC:  tensor(-54.3589, dtype=torch.float64)
FIN DQC:  tensor([ -4405.7811, -10939.0685], dtype=torch.float64)
SCF not converged.
SCF energy = -54.1815270434458
Energy PySCF:  -54.18152704344585
FIN PySCF:  tensor([-96134.5091, -67613.4090], dtype=torch.float64)


In [29]:
print("REVERSE: ", reverse_mode(ammonia, 1.0, 2.0))
print("FIN DQC: ", finite_difference(ammonia, 1.0, 2.0))
print("FIN PySCF: ", finite_difference_pyscf(ammonia, 1.0, 2.0))

Energy DQC:  tensor(-28.1712, dtype=torch.float64)
REVERSE:  tensor([ 7.4649, 12.7271], dtype=torch.float64)
Energy DQC:  tensor(-28.1712, dtype=torch.float64)
FIN DQC:  tensor([ 7.4649, 12.7271], dtype=torch.float64)
converged SCF energy = -28.1711795691315
Energy PySCF:  -28.171179569131475
FIN PySCF:  tensor([ 7.4649, 12.7270], dtype=torch.float64)


### Params a = 1.0, p = 3.0

In [30]:
print("REVERSE: ", reverse_mode(water, 1.0, 3.0))
print("FIN DQC: ", finite_difference(water, 1.0, 3.0))
print("FIN PySCF: ", finite_difference_pyscf(water, 1.0, 3.0))

Energy DQC:  tensor(-23.4711, dtype=torch.float64)
REVERSE:  tensor([ 0.7208, -0.5884], dtype=torch.float64)
Energy DQC:  tensor(-23.4711, dtype=torch.float64)
FIN DQC:  tensor([2.4030, 3.8163], dtype=torch.float64)
SCF not converged.
SCF energy = 7608.73432429647
Energy PySCF:  7608.734324296469
FIN PySCF:  tensor([-7.6324e+09,  6.8018e+07], dtype=torch.float64)


In [31]:
print("REVERSE: ", reverse_mode(h2_system, 1.0, 3.0))
print("FIN DQC: ", finite_difference(h2_system, 1.0, 3.0))
print("FIN PySCF: ", finite_difference_pyscf(h2_system, 1.0, 3.0))

Energy DQC:  tensor(-0.5084, dtype=torch.float64)
REVERSE:  tensor([ 0.0106, -0.0225], dtype=torch.float64)
Energy DQC:  tensor(-0.5084, dtype=torch.float64)
FIN DQC:  tensor([ 0.0106, -0.0225], dtype=torch.float64)
converged SCF energy = -0.508449635787078
Energy PySCF:  -0.5084496357870784
FIN PySCF:  tensor([ 0.0106, -0.0225], dtype=torch.float64)


In [32]:
print("REVERSE: ", reverse_mode(n2_system, 1.0, 3.0))
print("FIN DQC: ", finite_difference(n2_system, 1.0, 3.0))
print("FIN PySCF: ", finite_difference_pyscf(n2_system, 1.0, 3.0))

Energy DQC:  tensor(-33.5378, dtype=torch.float64)
REVERSE:  tensor([ 1.4839, -1.2046], dtype=torch.float64)
Energy DQC:  tensor(-33.5378, dtype=torch.float64)
FIN DQC:  tensor([4.8555, 9.3064], dtype=torch.float64)
converged SCF energy = -44.7751327106753
Energy PySCF:  -44.77513271067529
FIN PySCF:  tensor([5.0552, 2.9397], dtype=torch.float64)


In [33]:
print("REVERSE: ", reverse_mode(ammonia, 1.0, 3.0))
print("FIN DQC: ", finite_difference(ammonia, 1.0, 3.0))
print("FIN PYSCF: ", finite_difference_pyscf(ammonia, 1.0, 3.0))

/home/rolan/miniconda3/envs/noa/lib/python3.9/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=6.646e+02, |f|=6.592e+02 at iter 50
  warnings.warn(ConvergenceWarning(msg))


Energy DQC:  tensor(-17.3388, dtype=torch.float64)
REVERSE:  tensor([ 0.5239, -0.5451], dtype=torch.float64)
Energy DQC:  tensor(-17.3388, dtype=torch.float64)
FIN DQC:  tensor([2.0833, 3.6417], dtype=torch.float64)
SCF not converged.
SCF energy = -16.7893526102609
Energy PySCF:  -16.78935261026087
FIN PYSCF:  tensor([3.2849e+09, 1.1059e+09], dtype=torch.float64)


### Params a = -1.0, p = 2.0

In [34]:
print("REVERSE: ", reverse_mode(water, -1.0, 2.0))
print("FIN DQC: ", finite_difference(water, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(water, -1.0, 2.0))

Energy DQC:  tensor(-155.4981, dtype=torch.float64)
REVERSE:  tensor([  96.9847, -369.9213], dtype=torch.float64)
Energy DQC:  tensor(-155.4981, dtype=torch.float64)
FIN DQC:  tensor([  96.9847, -369.9230], dtype=torch.float64)
converged SCF energy = -155.498198456975
Energy PySCF:  -155.498198456975
FIN PYSCF:  tensor([  96.9848, -369.9231], dtype=torch.float64)


In [35]:
print("REVERSE: ", reverse_mode(h2_system, -1.0, 2.0))
print("FIN DQC: ", finite_difference(h2_system, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(h2_system, -1.0, 2.0))

Energy DQC:  tensor(-0.6463, dtype=torch.float64)
REVERSE:  tensor([0.1487, 0.3173], dtype=torch.float64)
Energy DQC:  tensor(-0.6463, dtype=torch.float64)
FIN DQC:  tensor([0.1487, 0.3173], dtype=torch.float64)
converged SCF energy = -0.646311203597606
Energy PySCF:  -0.6463112035976056
FIN PYSCF:  tensor([0.1487, 0.3173], dtype=torch.float64)


In [36]:
print("REVERSE: ", reverse_mode(n2_system, -1.0, 2.0))
print("FIN DQC: ", finite_difference(n2_system, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(n2_system, -1.0, 2.0))

Energy DQC:  tensor(-206.3187, dtype=torch.float64)
REVERSE:  tensor([ 113.7847, -413.4969], dtype=torch.float64)
Energy DQC:  tensor(-206.3187, dtype=torch.float64)
FIN DQC:  tensor([ 113.7847, -413.4986], dtype=torch.float64)
converged SCF energy = -206.318984748372
Energy PySCF:  -206.3189847483718
FIN PYSCF:  tensor([ 113.7850, -413.5003], dtype=torch.float64)


In [37]:
print("REVERSE: ", reverse_mode(ammonia, -1.0, 2.0))
print("FIN DQC: ", finite_difference(ammonia, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(ammonia, -1.0, 2.0))

Energy DQC:  tensor(-104.3939, dtype=torch.float64)
REVERSE:  tensor([  58.7489, -208.1992], dtype=torch.float64)
Energy DQC:  tensor(-104.3939, dtype=torch.float64)
FIN DQC:  tensor([  58.7489, -208.2000], dtype=torch.float64)
converged SCF energy = -104.394105081511
Energy PySCF:  -104.39410508151116
FIN PYSCF:  tensor([  58.7491, -208.2010], dtype=torch.float64)


In [38]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""

In [41]:
from math import pi
factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

In [42]:
print("REVERSE: ", reverse_mode(benzene_coords, factor, power))
print("FIN DQC: ", finite_difference(benzene_coords, factor, power))
print("FIN PySCF: ", finite_difference_pyscf(benzene_coords, factor, power))

Energy DQC:  tensor(-213.4143, dtype=torch.float64)
REVERSE:  tensor([ 48.7481, -40.5534], dtype=torch.float64)
Energy DQC:  tensor(-213.4143, dtype=torch.float64)
FIN DQC:  tensor([ 48.7481, -40.5536], dtype=torch.float64)
converged SCF energy = -213.414684215988
Energy PySCF:  -213.41468421598773
FIN PySCF:  tensor([ 48.7478, -40.5456], dtype=torch.float64)
